**Машинное обучение Урок 9**

Нужно реализовать rest api на базе flask в google colab.

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. Реализовать ноутбук с сервером
3. Реализовать ноутбук с клиентом

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv', delimiter=',')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [4]:
df['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df['toxic'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [7]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [8]:
features = ['id', 'comment_text']
target = 'toxic'

In [9]:
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [10]:
id = Pipeline([
                ('imputer', TextImputer('id', '')),
                ('selector', ColumnSelector(key='id')),
                ('tfidf', TfidfVectorizer())
            ])

comment_text = Pipeline([
                ('imputer', TextImputer('comment_text', '')),
                ('selector', ColumnSelector(key='comment_text')),
                ('tfidf', TfidfVectorizer())
            ])

feats = FeatureUnion([('id', id),
                      ('comment_text', comment_text)])

In [11]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

<ipython-input-7-2e5dbf19bf03>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.key] = X[self.key].fillna(self.value)


Wall time: 45.1 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('id',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='id',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='id')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('comment_text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='comment_text',
                                                                              value=''

In [12]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('id',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='id',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='id')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('comment_text',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='comment_text',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='comment_text')),
                                                  ('tfidf',
                               

In [16]:
import pickle

In [17]:
with open('pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [18]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [19]:
X_test.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus...",0,0,0,0,0,0
1,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...,0,0,0,0,0,0
2,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ...",0,0,0,0,0,0


In [20]:
with open('pipeline.pkl', 'rb') as in_strm:
    pipeline = pickle.load(in_strm)

In [21]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('id',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='id',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='id')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('comment_text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='comment_text',
                                                                              value=''

In [22]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [23]:
preds[:10]

array([0.21012667, 0.00932215, 0.10375638, 0.00581188, 0.00971549,
       0.01033128, 0.04732032, 0.04057704, 0.00148691, 0.04472439])

In [24]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.27399587295264427, F-Score=0.768, Precision=0.812, Recall=0.729


In [26]:
!pip install flask-ngrok

In [27]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [31]:
with open('pipeline.pkl', 'rb') as in_strm:
    model = pickle.load(in_strm)

In [32]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [33]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"


@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    id, comment_text = "", ""
    request_json = request.get_json()
    
    if request_json["id"]:
        id = request_json['id']
    
    if request_json["comment_text"]:
        comment_text = request_json['comment_text']
    
    print(id)  
    preds = model.predict_proba(pd.DataFrame({"id": [id],
                                              "comment_text": [comment_text],}))
    data["predictions"] = preds[:, 1][0]
    data["id"] = id
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9b7c-178-67-193-107.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [34]:
id_data, comment_text_data = ( 
    "9e5b8e8fc1ff2e95",
    "Carioca RFA \n\nThanks for your support on "
)

body = {
        'id': id_data, 
        'comment_text': comment_text_data
        }

In [35]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

9e5b8e8fc1ff2e95
OK


{'id': '9e5b8e8fc1ff2e95', 'predictions': 0.01607476666996328, 'success': True}

In [45]:
!pip install requests

In [50]:
import requests

import urllib.request
import json 

In [51]:
def send_json(x):
    id, comment_text = x
    body = {
        'id': id, 
        'comment_text': comment_text
        }
    myurl = 'http://9b7c-178-67-193-107.ngrok.io' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [52]:
data = (
     "9e5b8e8fc1ff2e95",
    "Carioca RFA \n\nThanks for your support on "
)

In [54]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [55]:
response = send_json(X_test[['id', 'comment_text']].iloc[0,:])
response

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [56]:
X_test[['id', 'comment_text']].head()

,id,comment_text
0,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus..."
1,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...
2,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ..."
3,5332799e706665a6,Pseudoscience category? \n\nI'm assuming that ...
4,dfa7d8f0b4366680,"(and if such phrase exists, it would be provid..."


In [57]:
N = 50

In [58]:
%%time
predictions = X_test[
                     ['id', 'comment_text']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [59]:
predictions.values[:5]

NameError: name 'predictions' is not defined

In [60]:
y_test.head()

,toxic
0,0
1,0
2,0
3,0
4,0


In [61]:
precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

NameError: name 'predictions' is not defined

In [62]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

NameError: name 'predictions' is not defined